## 8. The SMO algorithm

支持向量机的学习问题可以形式化为求解凸二次规划问题。这样的凸二次规划问题具有全局最优解，并且许多最优化算法可以用于对这一问题的求解，但是当训练样本容量很大时，这些算法往往变得非常低效，以致无法使用。所以，如何高效地实现支持向量机学习就成为一个重要的问题。目前人们已提出许多快速实现的算法。这里要介绍的就是 SMO 算法，SMO 是对 sequential minimal optimization（序列最小化优化） 的缩写，于1998年由 John Platt 在微软研究院提出。

SMO 算法要求解如下凸二次规划的对偶问题：

$$
\begin{align}
\max_\alpha&\quad\sum^m_{i=1}\alpha_i-\frac{1}{2}\sum^m_{i,j=1}\alpha_i\alpha_jy^{(i)}y^{(j)}K(x^{(i)},x^{(j)}) \\
s.t.&\quad\sum^m_{i=1}\alpha_iy^{(i)}=0 \\
&\quad 0\leq\alpha_i\leq C,\quad i=1,\cdots,m
\end{align}
$$

这个问题中，变量是拉格朗日乘子，一个变量 $\alpha_i$ 对应于一个样本点 $(x^{(i)},y^{(i)})$。变量的总数等于训练样本容量 $m$。

SMO 算法是一种启发式算法，其基本思路是：如果所有变量的解都满足此最优化问题的 KKT 条件，那么这个最优化问题的解就得到了。因为 KKT 条件是该最优化问题的充分必要条件。否则，选择两个变量，固定其他变量，针对这两个变量构建一个二次规划问题。这个二次规划问题关于这两个变量的解应该更接近原始二次规划问题的解，因为这会使得原始二次规划问题的目标函数值变得更小。更重要的是，这时子问题可以通过解析方法求解，这样就可以大大提高整个算法的计算速度。子问题有两个变量，一个是违反 KKT 条件最严重的那一个，另一个由约束条件自动确定。如此，SMO 算法将原问题不断分解为子问题并对子问题求解，从而达到求解原问题的目的。

注意，子问题的两个变量中只有一个是自由变量。假设 $\alpha_1,\alpha_2$ 为两个变量，$\alpha_3,\alpha_4,\cdots,\alpha_m$ 固定，那么由等式约束 $\sum^m_{i=1}\alpha_iy^{(i)}=0$ 可知：

$$
\alpha_1=-y^{(1)}\sum^m_{i=2}\alpha_iy^{(i)}
$$

所以如果 $\alpha_2$ 确定，那么 $\alpha_1$ 也随之确定。子问题中同时更新两个变量。

整个 SMO 算法包括两个部分：求解两个变量二次规划的解析方法和选择变量的启发式方法。

***

不失一般性，假设选择的两个变量是 $\alpha_1,\alpha_2$，其他变量 $\alpha_i(i=3,4,\cdots,m)$ 是固定的。于是 SMO 的最优化问题的子问题可以写成（这里把 $max$ 改成了 $min$，同时省略了不含 $\alpha_1,\alpha_2$ 的常数项）：

$$
\begin{align}
\min_{\alpha_1,\alpha_2}\quad W(\alpha_1,\alpha_2)
&=\sum^m_{i=1}\alpha_i\alpha_1y^{(i)}y^{(1)}K_{i1}+\sum^m_{i=1}\alpha_i\alpha_2y^{(i)}y^{(2)}K_{i2}-(\alpha_1+\alpha_2)\\
&=\frac{1}{2}K_{11}\alpha_1^2+\frac{1}{2}K_{22}\alpha_2^2+y^{(1)}y^{(2)}K_{12}\alpha_1\alpha_2-(\alpha_1+\alpha_2)+y^{(1)}\alpha_1\sum^m_{i=3}y^{(i)}\alpha_iK_{i1}+y^{(2)}\alpha_2\sum^m_{i=3}y^{(i)}\alpha_iK_{i2} \\
s.t.&\quad\alpha_1y^{(1)}+\alpha_2y^{(2)}=-\sum^m_{i=3}y^{(i)}\alpha_i=\zeta \\
&\quad 0\leq\alpha_i\leq C,i=1,2
\end{align}
$$

其中，$K_{ij}=K(x^{(i)},x^{(j)}),i,j=1,2,\cdots,m,\,\zeta$ 是常数。

首先，我们先分析约束条件，接着在此约束条件下求极小。

由于

然后，先不考虑不等式约束，即先求得沿着约束方向未经剪辑的 $\alpha_2$ 的最优解 $\alpha_2^{new,unc}$；然后再求剪辑后的 $\alpha_2$ 的解 $\alpha_2^{new}$。

记

$$
\begin{align}
g(x)&=\sum^m_{i=1}\alpha_iy^{(i)}K(x^{(i)},x)+b \\
E_i&=g(x^{(i)})-y^{(i)}=\left( \sum^m_{j=1}\alpha_jy^{(j)}K(x^{(j)},x^{(i)})+b \right)-y^{(i)},\quad i=1,2
\end{align}
$$

当 $i=1,2$ 时，$E_i$ 为函数 $g(x)$ 对输入 $x^{(i)}$ 的预测值与真实输出 $y^{(i)}$ 之差。

令

$$
v_i=\sum^m_{j=3}\alpha_jy^{(j)}K(x^{(i)},x^{(j)})=g(x^{(i)})-\sum^2_{j=1}\alpha_jy^{(j)}K(x^{(i)},x^{(j)})-b,\quad i=1,2
$$

目标函数可以写成：

$$
\begin{align}
W(\alpha_1,\alpha_2)=\frac{1}{2}K_{11}\alpha_1^2+\frac{1}{2}K_{22}\alpha_2^2+y^{(1)}y^{(2)}K_{12}\alpha_1\alpha_2-(\alpha_1+\alpha_2)+y^{(1)}v_1\alpha_1+y^{(2)}v_2\alpha_2
\end{align}
$$

由 $\alpha_1y^{(1)}=\zeta-\alpha_2y^{(2)}$ 及 $(y^{(i)})^2=1$，可将 $\alpha_1$ 表示为

$$
\alpha_1=(\zeta-\alpha_2y^{(2)})y^{(1)}
$$

将其代入目标函数，这样就得到只是 $\alpha_2$ 的函数的目标函数：

$$
W(\alpha_2)=\frac{1}{2}K_{11}(\zeta-\alpha_2y^{(2)})^2+\frac{1}{2}K_{22}\alpha_2^2+y^{(2)}K_{12}(\zeta-\alpha_2y^{(2)})\alpha_2-(\zeta-\alpha_2y^{(2)})y^{(1)}-\alpha_2+v_1(\zeta-\alpha_2y^{(2)})+y^{(2)}v_2\alpha_2
$$

对 $\alpha_2$ 求导：

$$
\frac{\partial W}{\partial \alpha_2}=K_{11}\alpha_2+K_{22}\alpha_2-2K_{12}\alpha_2-K_{11}\zeta y^{(2)}+K_{12}\zeta y^{(2)}+y^{(1)}y^{(2)}-1-v_1y^{(2)}+v_2y^{(2)}
$$

令其为零可得：

$$
\begin{align}
(K_{11}+K_{22}-2K_{12})\alpha_2
&=K_{11}\zeta y^{(2)}-K_{12}\zeta y^{(2)}-y^{(1)}y^{(2)}+1+v_1y^{(2)}-v_2y^{(2)} \\
&=y^{(2)}(y^{(2)}-y^{(1)}+\zeta K_{11}-\zeta K_{12}+v_1-v_2) \\
&=y^{(2)}\left[ y^{(2)}-y^{(1)}+\zeta K_{11}-\zeta K_{12}+\left( g(x^{(1)})-\sum^2_{j=1}\alpha_jy^{(j)}K_{1j}-b \right)-\left( g(x^{(2)})-\sum^2_{j=1}\alpha_jy^{(j)}K_{2j}-b \right) \right] \\
&=y^{(2)}\left( \zeta K_{11}-\zeta K_{12}+\alpha_1y^{(1)}K_{21}+\alpha_2y^{(2)}K_{22}-\alpha_1y^{(1)}K_{11}-\alpha_2y^{(2)}K_{12}+y^{(2)}-y^{(1)}+g(x^{(1)})-g(x^{(2)}) \right)
\end{align}
$$

将 $\zeta=\alpha_1^{old}y^{(1)}+\alpha_2^{old}y^{(2)}$ 代入，得到：

$$
\begin{align}
(K_{11}+K_{22}-2K_{12})\alpha_2^{new,unc}
&=y^{(2)}\left((K_{11}+K_{22}-2K_{12})\alpha_2^{old}y^{(2)}+y^{(2)}-y^{(1)}+g(x^{(1)})-g(x^{(2)})\right) \\
&=(K_{11}+K_{22}-2K_{12})\alpha_2^{old}+y^{(2)}(E_1-E_2)
\end{align}
$$

最后，令 $\eta=(K_{11}+K_{22}-2K_{12})=\|\phi(x^{(1)})-\phi(x^{(2)})\|^2$，可得：

$$
\alpha_2^{new,unc}=\alpha_2^{old}+\frac{y^{(2)}(E_1-E_2)}{\eta}
$$